## Aggregating Functions, Group BY, HAVING 

## Objectives

* Use SQL aggregation functions with GROUP BY
* Use HAVING for group filtering

## The Data

Below, we connect to a SQLite database using the Python `sqlite3` library ([documentation here](https://docs.python.org/3/library/sqlite3.html)):

In [1]:
import sqlite3 
import pandas as pd
conn = sqlite3.connect('data.sqlite')

The database that you've just connected to is the same database you have seen previously, containing data about orders, employees, etc. Here's an overview of the database:  

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png">

### 1.  What is the total amount of payments received from each customer?


In [5]:
query = ("""
         SELECT C.contactLastName, C.contactFirstName, C.customerNumber, SUM(P.amount) AS total_payments
         FROM customers AS C
         JOIN payments as P
         USING(customerNumber)
         GROUP BY C.customerNumber
         ORDER BY total_payments DESC
""")

pd.read_sql(query, conn)

,contactLastName,contactFirstName,customerNumber,total_payments
0,Freyre,Diego,141,715738.98
1,Nelson,Susan,124,584188.24
2,Ferguson,Peter,114,180585.07
3,Young,Jeff,151,177913.95
4,Natividad,Eric,148,156251.03
...,...,...,...,...
93,Cartrain,Pascale,381,29217.18
94,Ricotti,Franco,473,25358.32
95,Schmitt,Carine,103,22314.36
96,Taylor,Leslie,198,21554.26


### 2. What is the average number of orders placed by each customer?

In [17]:
query2 = ("""
          SELECT customerNumber, AVG(orderCount) AS avg_orders_per_customer
          FROM
          (SELECT customerNumber, COUNT(orderNumber) AS orderCount
          FROM orders
          GROUP BY customerNumber) AS customer_orders
          GROUP BY customerNumber;""")

pd.read_sql(query2, conn)

,customerNumber,avg_orders_per_customer
0,103,3.0
1,112,3.0
2,114,5.0
3,119,4.0
4,121,4.0
...,...,...
93,486,3.0
94,487,2.0
95,489,2.0
96,495,2.0


### 3. How many units of each product have been sold in total?

In [9]:
query3 = ("""
          SELECT productCode, SUM(quantityOrdered) AS total_quantity_sold
          FROM orderdetails
          GROUP BY productCode;""")

pd.read_sql(query3, conn)

,productCode,total_quantity_sold
0,S10_1678,1057
1,S10_1949,961
2,S10_2016,999
3,S10_4698,985
4,S10_4757,1030
...,...,...
104,S700_3505,952
105,S700_3962,896
106,S700_4002,1085
107,S72_1253,960


### 4. Which customers have made payments totaling more than $10,000?

In [15]:
query4 = ("""
          SELECT customerNumber, SUM(amount) AS total_payments
          FROM payments
          GROUP BY customerNumber
          HAVING SUM(amount) > 10000;
""")

pd.read_sql(query4, conn)

,customerNumber,total_payments
0,103,22314.36
1,112,80180.98
2,114,180585.07
3,119,116949.68
4,121,104224.79
...,...,...
92,486,77726.59
93,487,42570.37
94,489,29586.15
95,495,65541.74


### 5. Which customers have placed more than 5 orders?

In [14]:
query5 = ("""
          SELECT customerNumber, COUNT(orderNumber) AS total_orders
          FROM orders
          GROUP BY customerNumber
          HAVING COUNT(orderNumber) > 5;
""")

pd.read_sql(query5, conn)

,customerNumber,total_orders
0,124,17
1,141,26


### 6. Which products have sold more than 1000 units in total?

In [18]:
query6 = ("""
          SELECT productCode, SUM(quantityOrdered) AS total_quantity_sold
          FROM orderdetails
          GROUP BY productCode
          HAVING SUM(quantityOrdered) > 1000;
          """)

pd.read_sql(query6, conn)

,productCode,total_quantity_sold
0,S10_1678,1057
1,S10_4757,1030
2,S12_1108,1019
3,S12_2823,1028
4,S12_4473,1056
5,S18_1342,1111
6,S18_1662,1040
7,S18_2319,1053
8,S18_2949,1038
9,S18_3232,1808


### 7. Which customers have placed orders with a total value of more than $50,000?

In [25]:
query7 = ("""
          SELECT C.customerName, C.customerNumber,  SUM(ORS.priceEach * ORS.quantityOrdered) AS total_order_value
          FROM customers AS C
          JOIN orders AS OD
          USING(customerNumber)
          JOIN orderdetails AS ORS
          USING(orderNumber)
          GROUP BY C.customerNumber
          HAVING total_order_value > 50000
          ORDER BY total_order_value DESC
          """)

pd.read_sql(query7, conn)

,customerName,customerNumber,total_order_value
0,Euro+ Shopping Channel,141,820689.54
1,Mini Gifts Distributors Ltd.,124,591827.34
2,"Australian Collectors, Co.",114,180585.07
3,Muscle Machine Inc,151,177913.95
4,La Rochelle Gifts,119,158573.12
...,...,...,...
77,"Australian Collectables, Ltd",471,55866.02
78,Online Mini Collectables,204,55577.26
79,"Australian Gift Network, Co",333,55190.16
80,Mini Auto Werke,452,51059.99
